In [1]:
import arcpy
from dataclasses import dataclass
from archaic import FeatureClass

arcpy.env.workspace = r".data\canada.geodatabase"  # type: ignore


@dataclass(frozen=True)
class Mine:
    oid: int
    name_e: str
    type_e: str
    shape: arcpy.Multipoint


feature_class = FeatureClass[Mine]("main.mines_pt")

for feature in feature_class.read([1, 2, 7]):
    print(feature)

feature_class.info.__dict__

Mine(oid=1, name_e='Victor', type_e='Diamond', shape=<PointGeometry object at 0x260095f02e0[0x260095f0210]>)
Mine(oid=2, name_e='Musselwhite', type_e='Gold', shape=<PointGeometry object at 0x260095f0610[0x260095f0270]>)
Mine(oid=7, name_e='Bucko Lake', type_e='Nickel', shape=<PointGeometry object at 0x260095f02e0[0x260095f0210]>)


{'model': __main__.Mine,
 'dynamic': False,
 'catalog_path': '.data\\canada.geodatabase\\main.mines_pt',
 'oid_field_name': 'OBJECTID',
 'oid_property_name': 'oid',
 'has_parameterless_constructor': False,
 'properties': {'oid': 'OID@',
  'name_e': 'name_e',
  'type_e': 'type_e',
  'shape': 'SHAPE@'},
 'edit_properties': {'oid': 'OID@',
  'name_e': 'name_e',
  'type_e': 'type_e',
  'shape': 'SHAPE@'},
 '__orig_class__': archaic.info.Info[~T]}

In [ ]:
feature_class = FeatureClass("main.glaciers_p")

for feature in feature_class.read([1, 2, 7]):
    print(feature)

feature_class.info.__dict__